# K-Means

In [ ]:
from sklearn.cluster import KMeans
import tqdm

table = pd.read_csv("pjdata/dataIN_SEQ_30_WIN_04.csv")
# table = pd.DataFrame(data=sequences, columns=col)

max_n = 100
interval = 10
wss = []
sis = []
for k in tqdm.tqdm(range(10, max_n+1, interval)):
    kmeans = KMeans(n_clusters=k, random_state=1234)
    kmeans.fit(table.iloc[:,1:].values)
    wss = np.append(wss, kmeans.inertia_)

fig, ax = plt.subplots(figsize=(7,4))
line = ax.plot(range(10, max_n+1, interval), wss, 'ro--', label='SSE')
ax.set_ylim(wss.min()*0.95, wss.max()*1.05)
ax.set_xticks(range(10, max_n+1, interval))
ax.set_xlabel('cluster num')
ax.set_ylabel('SSE')
labels = [l.get_label() for l in line]
plt.legend(line, labels)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import tqdm
import pickle

inout_flg = "" # ""
label_dicts = {}
iterations = []
for SEQ in ("10","20","30","40","50"):
    for WIN in ("03","04","05","06"):
        for CLUSTER_NUM in (20,30,50,100):
            iterations.append((SEQ, WIN, CLUSTER_NUM))
            
for SEQ, WIN, CLUSTER_NUM in tqdm.tqdm(iterations):
    cur = f"SEQ_{SEQ}_WIN_{WIN}_CLS_{str(CLUSTER_NUM)}"
    table = pd.read_csv(f"pjdata/data{inout_flg}_SEQ_{SEQ}_WIN_{WIN}.csv")
    kmeans = KMeans(n_clusters=CLUSTER_NUM, random_state=1234)
    kmeans.fit(table.iloc[:,1:].values)
    table['cluster'] = kmeans.predict(table.iloc[:,1:])
    result = table.groupby('cluster').actions.mean()

    print(cur)
    print(result[((result >= 1.1) & (result <= 1.9)) | (result >= 2.1)])
    print()

    label_lists = []
    tmp = list(result[((result >= 1.1) & (result <= 1.9)) | (result >= 2.1)].index)
    if len(tmp) > 0:
        for cls in tmp:
            label_lists += list(table[table.cluster == cls].index)
    label_lists.sort()
    label_dicts[cur] = label_lists

with open(f"pjdata/kmeans{inout_flg}_results.pickle", "wb") as f:
    pickle.dump(label_dicts, f)

# dbscan

In [ ]:
import gc
import distro
import re
import subprocess
import tables
from tempfile import NamedTemporaryFile
import time
import warnings

import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import kneighbors_graph 
from sklearn.utils import check_random_state

np.seterr(invalid = 'ignore')
warnings.filterwarnings('ignore', category = DeprecationWarning)
__all__ = ['DBSCAN', 'load', 'shoot']

def memory():
    mem_info = {}
    if distro.linux_distribution()[0]:
        with open('/proc/meminfo') as file:
            c = 0
            for line in file:
                lst = line.split()
                if str(lst[0]) == 'MemTotal:':
                    mem_info['total'] = int(lst[1])
                elif str(lst[0]) in ('MemFree:', 'Buffers:', 'Cached:'):
                    c += int(lst[1])
            mem_info['free'] = c
            mem_info['used'] = (mem_info['total']) - c
    elif distro.mac_ver()[0]:
        ps = subprocess.Popen(['ps', '-caxm', '-orss,comm'], stdout=subprocess.PIPE).communicate()[0]
        vm = subprocess.Popen(['vm_stat'], stdout=subprocess.PIPE).communicate()[0]
        
        # Iterate processes
        process_lines = ps.split('\n')
        sep = re.compile('[\s]+')
        rss_total = 0  # kB
        for row in range(1, len(process_lines)):
            row_text = process_lines[row].strip()
            row_elements = sep.split(row_text)
            try:
                rss = float(row_elements[0]) * 1024
            except:
                rss = 0  # ignore...
            rss_total += rss
            
        # Process vm_stat
        vm_lines = vm.split('\n')
        sep = re.compile(':[\s]+')
        vm_stats = {}
        for row in range(1, len(vm_lines) - 2):
            row_text = vm_lines[row].strip()
            row_elements = sep.split(row_text)
            vm_stats[(row_elements[0])] = int(row_elements[1].strip('\.')) * 4096

        mem_info['total'] = rss_total
        mem_info['used'] = vm_stats["Pages active"]
        mem_info['free'] = vm_stats["Pages free"]
    else:
        raise('Unsupported Operating System.\n')
        exit(1)

    return mem_info


def get_chunk_size(N, n):
    mem_free = memory()['free']
    if mem_free > 60000000:
        chunks_size = int(((mem_free - 10000000) * 1000) / (4 * n * N))
        return chunks_size
    elif mem_free > 40000000:
        chunks_size = int(((mem_free - 7000000) * 1000) / (4 * n * N))
        return chunks_size
    elif mem_free > 14000000:
        chunks_size = int(((mem_free - 2000000) * 1000) / (4 * n * N))
        return chunks_size
    elif mem_free > 8000000:
        chunks_size = int(((mem_free - 1400000) * 1000) / (4 * n * N))
        return chunks_size
    elif mem_free > 2000000:
        chunks_size = int(((mem_free - 900000) * 1000) / (4 * n * N))
        return chunks_size
    elif mem_free > 1000000:
        chunks_size = int(((mem_free - 400000) * 1000) / (4 * n * N))
        return chunks_size
    else:
        raise MemoryError("\nERROR: DBSCAN_multiplex @ get_chunk_size:\n"
                          "this machine does not have enough free memory "
                          "to perform the remaining computations.\n")


def load(hdf5_file_name, data, minPts, eps = None, quantile = 50, subsamples_matrix = None, samples_weights = None, 
metric = 'minkowski', p = 2, verbose = True):
    
    data = np.array(data, copy = False)
    if data.ndim > 2:
        raise ValueError("\nERROR: DBSCAN_multiplex @ load:\n" 
                         "the data array is of dimension %d. Please provide a two-dimensional "
                         "array instead.\n" % data.ndim)

    if subsamples_matrix is None:
        subsamples_matrix = np.arange(data.shape[0], dtype = int)
        subsamples_matrix = subsamples_matrix.reshape(1, -1)
    else:
        subsamples_matrix = np.array(subsamples_matrix, copy = False)

    if subsamples_matrix.ndim > 2:
        raise ValueError("\nERROR: DBSCAN_multiplex @ load:\n"
                         "the array of subsampled indices is of dimension %d. "
                         "Please provide a two-dimensional array instead.\n" % subsamples_matrix.ndim)
    if (data.dtype.char in np.typecodes['AllFloat'] and not np.isfinite(data.sum()) and not np.all(np.isfinite(data))):
        raise ValueError('\nERROR: DBSCAN_multiplex @ load:\n'
                         'the data vector contains at least one infinite or NaN entry.\n')
    if (subsamples_matrix.dtype.type is np.int_ and not np.isfinite(subsamples_matrix.sum()) and not np.all(np.isfinite(subsamples_matrix))):
        raise ValueError('\nERROR: DBSCAN_multiplex @ load:\n' 
                         'the array of subsampled indices contains at least one infinite or NaN entry.\n')
    if not np.all(subsamples_matrix >= 0):
        raise ValueError('\nERROR: DBSCAN_multiplex @ load:\n'
                         'the sampled indices should all be positive integers.\n') 

    N_samples = data.shape[0]
    N_runs, N_subsamples = subsamples_matrix.shape

    if N_subsamples > N_samples:
        raise ValueError('\nERROR: DBSCAN_multiplex @ load:\n'
                         'the number of sampled indices cannot exceed the total number of samples in the whole data-set.\n')
        
    for i in range(N_runs):
        subsamples_matrix[i] = np.unique(subsamples_matrix[i])
        
    if not isinstance(minPts, int):
        raise TypeError("\nERROR: DBSCAN_multiplex @ load:\n"
                        "the parameter 'minPts' must be an integer.\n")
    if minPts < 2:
        raise ValueError("\nERROR: DBSCAN_multiplex @ load:\n"
                         "the value of 'minPts' must be larger than 1.\n")        

    if eps is None:
        if verbose:
            print(("INFO: DBSCAN_multiplex @ load:\n"
                  "starting the determination of an appropriate value of 'eps' for this data-set"
                  " and for the other parameter of the DBSCAN algorithm set to {minPts}.\n"
                  "This might take a while.".format(**locals())))

        beg_eps = time.time()
        quantile = np.rint(quantile)
        quantile = np.clip(quantile, 0, 100)
        k_distances = kneighbors_graph(data, minPts, mode = 'distance', metric = metric, p = p).data
 
        radii = np.zeros(N_samples, dtype = float)
        for i in range(0, minPts):
            radii = np.maximum(radii, k_distances[i::minPts]) 

        if quantile == 50:     
            eps = round(np.median(radii, overwrite_input = True), 4)
        else:
            eps = round(np.percentile(radii, quantile), 4)
            
        end_eps = time.time()
        if verbose:
            print(("\nINFO: DBSCAN_multiplex @ load:\n"
                  "done with evaluating parameter 'eps' from the data-set provided."
                  " This took {} seconds. Value of epsilon: {}.".format(round(end_eps - beg_eps, 4), eps)))
    else:
        if not (isinstance(eps, float) or isinstance(eps, int)):
            raise ValueError("\nERROR: DBSCAN_multiplex @ load:\n"
                             "please provide a numeric value for the radius 'eps'.\n")
        if not eps > 0.0:
            raise ValueError("\nERROR: DBSCAN_multiplex @ load:\n"
                             "the radius 'eps' must be positive.\n")
        eps = round(eps, 4)

    if verbose:
        print(("\nINFO: DBSCAN_multiplex @ load:\n"
             "identifying the neighbors within an hypersphere of radius {eps} around each sample,"
             " while at the same time evaluating the number of epsilon-neighbors for each sample.\n"
             "This might take a fair amount of time.".format(**locals())))

    beg_neigh = time.time()

    fileh = tables.open_file(hdf5_file_name, mode = 'r+')
    DBSCAN_group = fileh.create_group(fileh.root, 'DBSCAN_group')
    neighborhoods_indices = fileh.create_earray(DBSCAN_group, 'neighborhoods_indices', tables.Int32Atom(), (0,), 
                                                'Indices array for sparse matrix of neighborhoods', 
                                                expectedrows = int((N_samples ** 2) / 50))

    neighborhoods_indptr = np.zeros(1, dtype = np.int64)
    neighbors_counts = fileh.create_carray(DBSCAN_group, 'neighbors_counts', tables.Int32Atom(), (N_runs, N_samples), 
                                           'Array of the number of neighbors around each sample of a set of subsampled points', 
                                           filters = None)   
    chunks_size = get_chunk_size(N_samples, 3)
    
    for i in range(0, N_samples, chunks_size):
        chunk = data[i:min(i + chunks_size, N_samples)]
        D = pairwise_distances(chunk, data, metric = metric, p = p, n_jobs = 1)
        D = (D <= eps)
        if samples_weights is None:
            for run in range(N_runs):
                x = subsamples_matrix[run]
                M = np.take(D, x, axis = 1)
                legit_rows = np.intersect1d(i + np.arange(min(chunks_size, N_samples - i)), x, assume_unique = True)
                M = np.take(M, legit_rows - i, axis = 0)
                neighbors_counts[run, legit_rows] = M.sum(axis = 1)
                del M
        else:
            for run in range(N_runs):
                x = subsamples_matrix[run]
                M = np.take(D, x, axis = 1)
                legit_rows = np.intersect1d(i + np.arange(min(chunks_size, N_samples - i)), x, assume_unique = True)
                M = np.take(M, legit_rows - i, axis = 0)
                neighbors_counts[run, legit_rows] = np.array([np.sum(samples_weights[x[row]]) for row in M])
                del M

        candidates = np.where(D == True)
        del D
        neighborhoods_indices.append(candidates[1])
        _, nbr = np.unique(candidates[0], return_counts = True)
        counts = np.cumsum(nbr) + neighborhoods_indptr[-1]

        del candidates
        neighborhoods_indptr = np.append(neighborhoods_indptr, counts)

    fileh.create_carray(DBSCAN_group, 'neighborhoods_indptr', tables.Int64Atom(), (N_samples + 1,), 
                        'Array of cumulative number of column indices for each row', filters = None)
    fileh.root.DBSCAN_group.neighborhoods_indptr[:] = neighborhoods_indptr[:]
    fileh.create_carray(DBSCAN_group, 'subsamples_matrix', tables.Int32Atom(), (N_runs, N_subsamples), 
                        'Array of subsamples indices', filters = None)
    fileh.root.DBSCAN_group.subsamples_matrix[:] = subsamples_matrix[:]
    fileh.close()

    end_neigh = time.time()

    if verbose:
        print(("\nINFO: DBSCAN_multiplex @ load:\n"
              "done with the neighborhoods. This step took {} seconds.".format(round(end_neigh - beg_neigh, 4))))
    gc.collect()
    
    return eps

def shoot(hdf5_file_name, minPts, sample_ID = 0, random_state = None, verbose = True):
    fileh = tables.open_file(hdf5_file_name, mode = 'r+')

    neighborhoods_indices = fileh.root.DBSCAN_group.neighborhoods_indices
    neighborhoods_indptr = fileh.root.DBSCAN_group.neighborhoods_indptr[:]

    neighbors_counts = fileh.root.DBSCAN_group.neighbors_counts[sample_ID]
    subsampled_indices = fileh.root.DBSCAN_group.subsamples_matrix[sample_ID]

    N_samples = neighborhoods_indptr.size - 1
    N_runs, N_subsamples = fileh.root.DBSCAN_group.subsamples_matrix.shape

    if not isinstance(sample_ID, int):
        raise ValueError("\nERROR: DBSCAN_multiplex @ shoot:\n"
                         "'sample_ID' must be an integer identifying the set of subsampled indices "
                         "on which to perform DBSCAN clustering\n")    

    if (sample_ID < 0) or (sample_ID >= N_runs):
        raise ValueError("\nERROR: DBSCAN_multiplex @ shoot:\n"
                 "'sample_ID' must belong to the interval [0; {}].\n".format(N_runs - 1))
      
    labels = np.full(N_samples, -2, dtype = int)
    labels[subsampled_indices] = - 1
    
    random_state = check_random_state(random_state)
    core_samples = np.flatnonzero(neighbors_counts >= minPts)
    index_order = np.take(core_samples, random_state.permutation(core_samples.size))
    cluster_ID = 0

    for index in index_order:
        if labels[index] not in {-1, -2}:
            continue

        labels[index] = cluster_ID
        candidates = [index]
        while len(candidates) > 0:
            candidate_neighbors = np.zeros(0, dtype = np.int32)
            for k in candidates:
                candidate_neighbors = np.append(candidate_neighbors, neighborhoods_indices[neighborhoods_indptr[k]: neighborhoods_indptr[k+1]])
                candidate_neighbors = np.unique(candidate_neighbors)
            candidate_neighbors = np.intersect1d(candidate_neighbors, subsampled_indices, assume_unique = True)
            not_noise_anymore = np.compress(np.take(labels, candidate_neighbors) == -1, candidate_neighbors)
            labels[not_noise_anymore] = cluster_ID
            candidates = np.intersect1d(not_noise_anymore, core_samples, assume_unique = True) 
        cluster_ID += 1

    fileh.close()
    gc.collect()
    return core_samples, labels


def DBSCAN(data, minPts, eps = None, quantile = 50, subsamples_matrix = None, samples_weights = None, 
metric = 'minkowski', p = 2, verbose = True):
        
    assert isinstance(minPts, int) or type(minPts) is np.int_
    assert minPts > 1

    if subsamples_matrix is None:
        subsamples_matrix = np.arange(data.shape[0], dtype = int)
        subsamples_matrix = subsamples_matrix.reshape(1, -1)
    else:
        subsamples_matrix = np.array(subsamples_matrix, copy = False)

    N_runs = subsamples_matrix.shape[0]
    N_samples = data.shape[0]

    labels_matrix = np.zeros((N_runs, N_samples), dtype = int)

    with NamedTemporaryFile('w', suffix = '.h5', delete = True, dir = './') as f:
        eps = load(f.name, data, minPts, eps, quantile, subsamples_matrix, samples_weights, metric, p, verbose)

        for run in range(N_runs):
            _, labels = shoot(f.name, minPts, sample_ID = run, verbose = verbose)
            labels_matrix[run] = labels

    return eps, labels_matrix

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

inout_flg = "IN" # ""
label_dicts = {}
iterations = []
for SEQ in ("10","20","30","40","50"):
    for WIN in ("03","04","05","06"):
            iterations.append((SEQ, WIN))

for SEQ, WIN in tqdm.tqdm(iterations):
    table = pd.read_csv(f"pjdata/data{inout_flg}_SEQ_{SEQ}_WIN_{WIN}.csv")
    SIZE = table.shape[0]
    data = table.iloc[:SIZE,:]

    N_iterations = 1
    N_sub = data.shape[0]
    subsamples_matrix = np.zeros((N_iterations, N_sub), dtype=int)

    for i in range(N_iterations): 
        subsamples_matrix[i] = np.random.choice(data.shape[0], N_sub, replace=False)

    eps, labels_matrix = DBSCAN(data.iloc[:,1:].values, minPts=5, subsamples_matrix=subsamples_matrix, verbose=False)

    data['cluster'] = labels_matrix[0]
    jn = (data[data.cluster != -1].groupby('cluster').actions.count() > 0).reset_index()
    jn.columns = ['cluster', 'flg']
    data = data.merge(jn, on='cluster', how='left').fillna(False)
    result = data[data.flg==True].groupby('cluster').actions.mean()
    print(cur)
    print(data.cluster.value_counts())
    print(result[((result >= 1.1) & (result <= 1.9)) | (result >= 2.1)])
    print("---------------------------------------------------")
    print()

    label_lists = []
    tmp = result[((result >= 1.1) & (result <= 1.9) | (result >= 2.1))].index
    if len(tmp) > 0:
        for cls in tmp:
            label_lists += list(data[data.cluster == cls].index)
    label_lists.sort()
    label_dicts[cur] = label_lists

with open(f"pjdata/dbscan{inout_flg}_results.pickle", "wb") as f:
    pickle.dump(label_dicts, f)

# DBSCAN - sklearn

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
import tqdm

iterations = []
for SEQ in ("30","40"):
    for WIN in ("03","06"):
        for EPS in (2.0, 2.25, 2.5):
            for MIN_SAMPLE in (5, 10):
                iterations.append((SEQ, WIN, EPS, MIN_SAMPLE))

for SEQ, WIN, EPS, MIN_SAMPLE in tqdm.tqdm(iterations):
    cur = f"SEQ_{SEQ}_WIN_{WIN}_MIN_{str(MIN_SAMPLE).zfill(2)}_EPS_{str(EPS)}"
    table = pd.read_csv(f"pjdata/data_SEQ_{SEQ}_WIN_{WIN}.csv")
    dbscan = DBSCAN(eps=EPS, min_samples=MIN_SAMPLE)
    dbscan.fit(table.iloc[:,1:].values)
    table['cluster'] = dbscan.labels_
    
    print(cur)
    print(table.cluster.value_counts())
    print()